In [223]:
import requests
import datetime
import pprint
import sqlite3
import json
import pandas as pd

In [224]:
last_week = (datetime.date.today() - datetime.timedelta(days=7)).strftime("%Y-%m-%d")
link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={last_week}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
conn = sqlite3.connect('NASA_NEO.db')

Exploration of ETL

In [225]:
# for date, neo_list in neo_data.items():
#     for neo in neo_list:
#         cursor.execute('''
#                     INSERT INTO neo VALUES (?, ?, ?, ?, ?, ?)
#                     ''', (
#                         neo['id'],
#                         neo['name'],
#                         neo['is_potentially_hazardous_asteroid'],
#                         neo['is_sentry_object'],
#                         neo['close_approach_data'],
#                         neo['estimated_diameter']
#                     ))
# conn.commit()

ELT

In [226]:
def load_data():
    response = requests.get(link)
    data = response.json()
    neo_data = data['near_earth_objects']
  
    #pprint.pprint(neo_data)

    cursor = conn.cursor()

    cursor.executescript('''

        DROP TABLE IF EXISTS neo;

        CREATE TABLE neo (
                        id text,
                        data json
        )
    ''')

    for date, neo_list in neo_data.items():
        for neo in neo_list:
            cursor.execute('INSERT INTO neo VALUES (?, ?)',
                        (neo['id'], json.dumps(neo)))
    conn.commit()

Exploration of SQL transformation

In [227]:
# df = pd.read_sql_query('''
#                        select json_extract(data, "$.name") as name,
#                        json_extract(data, "$.id") as ID,
#                        json_extract(data, "$.is_potentially_hazardous_asteroid") as potentially_hazardous
#                        from neo;
#                        ''', conn)

# df["potentially_hazardous"] = df["potentially_hazardous"].astype(bool)

Pandas transformation

In [228]:
df = pd.read_sql_query('''SELECT data FROM neo''', conn)

df['data'] = df['data'].apply(json.loads)

json_df = pd.json_normalize(df['data'])

json_df

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,3384067,3384067,(2007 RU9),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3384067,20.700,True,"[{'close_approach_date': '2023-09-28', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3384067?ap...,0.192555,0.430566,192.555078,430.566244,0.119648,0.267541,631.742403,1412.618957
1,3581506,3581506,(2011 TB4),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3581506,25.400,False,"[{'close_approach_date': '2023-09-28', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3581506?ap...,0.022108,0.049436,22.108281,49.435619,0.013737,0.030718,72.533733,162.190357
2,3608565,3608565,(2012 RH10),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3608565,22.140,False,"[{'close_approach_date': '2023-09-28', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3608565?ap...,0.099210,0.221840,99.209892,221.840062,0.061646,0.137845,325.491782,727.821750
3,3648878,3648878,(2013 TG6),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3648878,26.600,False,"[{'close_approach_date': '2023-09-28', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3648878?ap...,0.012722,0.028447,12.721988,28.447230,0.007905,0.017676,41.738807,93.330809
4,3690061,3690061,(2014 SH144),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3690061,25.000,False,"[{'close_approach_date': '2023-09-28', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3690061?ap...,0.026580,0.059435,26.580000,59.434687,0.016516,0.036931,87.204727,194.995698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,54202343,54202343,(2021 TQ1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54202343,28.090,False,"[{'close_approach_date': '2023-09-21', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54202343?a...,0.006406,0.014323,6.405529,14.323197,0.003980,0.008900,21.015515,46.992119
89,54244535,54244535,(2022 CT),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54244535,25.730,False,"[{'close_approach_date': '2023-09-21', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54244535?a...,0.018991,0.042466,18.991312,42.465865,0.011801,0.026387,62.307457,139.323710
90,54310136,54310136,(2022 SD29),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54310136,26.480,False,"[{'close_approach_date': '2023-09-21', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54310136?a...,0.013445,0.030064,13.444820,30.063530,0.008354,0.018681,44.110302,98.633633
91,54359399,54359399,(2023 KK1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54359399,20.200,True,"[{'close_approach_date': '2023-09-21', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54359399?a...,0.242412,0.542051,242.412481,542.050786,0.150628,0.336815,795.316564,1778.381902


Extracting nested close approach data

In [229]:
json_df['close_approach_data'][0]

[{'close_approach_date': '2023-09-28',
  'close_approach_date_full': '2023-Sep-28 15:52',
  'epoch_date_close_approach': 1695916320000,
  'relative_velocity': {'kilometers_per_second': '21.2534376859',
   'kilometers_per_hour': '76512.3756692011',
   'miles_per_hour': '47541.8147595656'},
  'miss_distance': {'astronomical': '0.1604399752',
   'lunar': '62.4111503528',
   'kilometers': '24001478.552772824',
   'miles': '14913827.2093360112'},
  'orbiting_body': 'Earth'}]

Converting list of dict to dict

In [230]:
for n in json_df.index.values:
        json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}

json_df

C:\Users\Jack\AppData\Local\Temp\ipykernel_14044\523775224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}


,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,3384067,3384067,(2007 RU9),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3384067,20.700,True,"{'close_approach_date': '2023-09-28', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3384067?ap...,0.192555,0.430566,192.555078,430.566244,0.119648,0.267541,631.742403,1412.618957
1,3581506,3581506,(2011 TB4),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3581506,25.400,False,"{'close_approach_date': '2023-09-28', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3581506?ap...,0.022108,0.049436,22.108281,49.435619,0.013737,0.030718,72.533733,162.190357
2,3608565,3608565,(2012 RH10),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3608565,22.140,False,"{'close_approach_date': '2023-09-28', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3608565?ap...,0.099210,0.221840,99.209892,221.840062,0.061646,0.137845,325.491782,727.821750
3,3648878,3648878,(2013 TG6),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3648878,26.600,False,"{'close_approach_date': '2023-09-28', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3648878?ap...,0.012722,0.028447,12.721988,28.447230,0.007905,0.017676,41.738807,93.330809
4,3690061,3690061,(2014 SH144),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3690061,25.000,False,"{'close_approach_date': '2023-09-28', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3690061?ap...,0.026580,0.059435,26.580000,59.434687,0.016516,0.036931,87.204727,194.995698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,54202343,54202343,(2021 TQ1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54202343,28.090,False,"{'close_approach_date': '2023-09-21', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54202343?a...,0.006406,0.014323,6.405529,14.323197,0.003980,0.008900,21.015515,46.992119
89,54244535,54244535,(2022 CT),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54244535,25.730,False,"{'close_approach_date': '2023-09-21', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54244535?a...,0.018991,0.042466,18.991312,42.465865,0.011801,0.026387,62.307457,139.323710
90,54310136,54310136,(2022 SD29),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54310136,26.480,False,"{'close_approach_date': '2023-09-21', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54310136?a...,0.013445,0.030064,13.444820,30.063530,0.008354,0.018681,44.110302,98.633633
91,54359399,54359399,(2023 KK1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54359399,20.200,True,"{'close_approach_date': '2023-09-21', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54359399?a...,0.242412,0.542051,242.412481,542.050786,0.150628,0.336815,795.316564,1778.381902


In [231]:
cad_df = pd.json_normalize(json_df['close_approach_data'])

cad_df

,close_approach_date,close_approach_date_full,epoch_date_close_approach,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,relative_velocity.miles_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,miss_distance.miles
0,2023-09-28,2023-Sep-28 15:52,1695916320000,Earth,21.2534376859,76512.3756692011,47541.8147595656,0.1604399752,62.4111503528,24001478.552772824,14913827.2093360112
1,2023-09-28,2023-Sep-28 21:43,1695937380000,Earth,19.6051964197,70578.7071110284,43854.8638712943,0.2179460329,84.7810067981,32604262.296789923,20259349.1527220174
2,2023-09-28,2023-Sep-28 00:02,1695859320000,Earth,10.2005587063,36722.0113425569,22817.629770046,0.2989174969,116.2789062941,44717420.841971603,27786116.8518420014
3,2023-09-28,2023-Sep-28 21:33,1695936780000,Earth,4.1477046929,14931.7368946125,9278.0006276552,0.0091324629,3.5525280681,1366196.997694023,848915.4496345974
4,2023-09-28,2023-Sep-28 21:46,1695937560000,Earth,12.5398583222,45143.4899599524,28050.4090809493,0.0956959945,37.2257418605,14315916.944731715,8895498.299719067
...,...,...,...,...,...,...,...,...,...,...,...
88,2023-09-21,2023-Sep-21 05:58,1695275880000,Earth,20.3897653573,73403.1552863347,45609.8661277784,0.4971154154,193.3778965906,74367407.288005198,46209764.1135318124
89,2023-09-21,2023-Sep-21 06:28,1695277680000,Earth,8.5561768174,30802.2365427628,19139.3119229184,0.4900134095,190.6152162955,73304962.332637765,45549591.430880557
90,2023-09-21,2023-Sep-21 05:47,1695275220000,Earth,10.5189116588,37868.0819718337,23529.7534896652,0.217857528,84.746578392,32591022.15226536,20251122.108407568
91,2023-09-21,2023-Sep-21 01:09,1695258540000,Earth,13.3678083073,48124.1099061374,29902.4504025108,0.1966111821,76.4817498369,29412614.060342127,18276150.9007182726


In [232]:
df = pd.concat([json_df, cad_df, df], axis=1)

df

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,...,epoch_date_close_approach,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,relative_velocity.miles_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,miss_distance.miles,data
0,3384067,3384067,(2007 RU9),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3384067,20.700,True,"{'close_approach_date': '2023-09-28', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3384067?ap...,0.192555,...,1695916320000,Earth,21.2534376859,76512.3756692011,47541.8147595656,0.1604399752,62.4111503528,24001478.552772824,14913827.2093360112,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,3581506,3581506,(2011 TB4),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3581506,25.400,False,"{'close_approach_date': '2023-09-28', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3581506?ap...,0.022108,...,1695937380000,Earth,19.6051964197,70578.7071110284,43854.8638712943,0.2179460329,84.7810067981,32604262.296789923,20259349.1527220174,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3608565,3608565,(2012 RH10),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3608565,22.140,False,"{'close_approach_date': '2023-09-28', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3608565?ap...,0.099210,...,1695859320000,Earth,10.2005587063,36722.0113425569,22817.629770046,0.2989174969,116.2789062941,44717420.841971603,27786116.8518420014,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3648878,3648878,(2013 TG6),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3648878,26.600,False,"{'close_approach_date': '2023-09-28', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3648878?ap...,0.012722,...,1695936780000,Earth,4.1477046929,14931.7368946125,9278.0006276552,0.0091324629,3.5525280681,1366196.997694023,848915.4496345974,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3690061,3690061,(2014 SH144),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3690061,25.000,False,"{'close_approach_date': '2023-09-28', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3690061?ap...,0.026580,...,1695937560000,Earth,12.5398583222,45143.4899599524,28050.4090809493,0.0956959945,37.2257418605,14315916.944731715,8895498.299719067,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,54202343,54202343,(2021 TQ1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54202343,28.090,False,"{'close_approach_date': '2023-09-21', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54202343?a...,0.006406,...,1695275880000,Earth,20.3897653573,73403.1552863347,45609.8661277784,0.4971154154,193.3778965906,74367407.288005198,46209764.1135318124,{'links': {'self': 'http://api.nasa.gov/neo/re...
89,54244535,54244535,(2022 CT),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54244535,25.730,False,"{'close_approach_date': '2023-09-21', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54244535?a...,0.018991,...,1695277680000,Earth,8.5561768174,30802.2365427628,19139.3119229184,0.4900134095,190.6152162955,73304962.332637765,45549591.430880557,{'links': {'self': 'http://api.nasa.gov/neo/re...
90,54310136,54310136,(2022 SD29),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54310136,26.480,False,"{'close_approach_date': '2023-09-21', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54310136?a...,0.013445,...,1695275220000,Earth,10.5189116588,37868.0819718337,23529.7534896652,0.217857528,84.746578392,32591022.15226536,20251122.108407568,{'links': {'self': 'http://api.nasa.gov/neo/re...
91,54359399,54359399,(2023 KK1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54359399,20.200,True,"{'close_approach_date': '2023-09-21', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54359399?a...,0.242412,...,1695258540000,Earth,13.3678083073,48124.1099061374,29902.4504025108,0.1966111821,76.4817498369,29412614.060

Cleaning up df

In [233]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 29 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   id                                                    93 non-null     object 
 1   neo_reference_id                                      93 non-null     object 
 2   name                                                  93 non-null     object 
 3   nasa_jpl_url                                          93 non-null     object 
 4   absolute_magnitude_h                                  93 non-null     float64
 5   is_potentially_hazardous_asteroid                     93 non-null     bool   
 6   close_approach_data                                   93 non-null     object 
 7   is_sentry_object                                      93 non-null     bool   
 8   links.self                                            93 non-n

In [234]:
df = df.drop(['neo_reference_id', 'close_approach_data', 'links.self',
        'estimated_diameter.kilometers.estimated_diameter_min',
        'estimated_diameter.kilometers.estimated_diameter_max',
        'estimated_diameter.miles.estimated_diameter_min',
        'estimated_diameter.miles.estimated_diameter_max',
        'estimated_diameter.feet.estimated_diameter_min',
        'estimated_diameter.feet.estimated_diameter_max',
        'close_approach_date', 'epoch_date_close_approach',
        'relative_velocity.miles_per_hour',
        'miss_distance.miles'], axis=1
        )

df

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,close_approach_date_full,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,data
0,3384067,(2007 RU9),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3384067,20.700,True,False,192.555078,430.566244,2023-Sep-28 15:52,Earth,21.2534376859,76512.3756692011,0.1604399752,62.4111503528,24001478.552772824,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,3581506,(2011 TB4),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3581506,25.400,False,False,22.108281,49.435619,2023-Sep-28 21:43,Earth,19.6051964197,70578.7071110284,0.2179460329,84.7810067981,32604262.296789923,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3608565,(2012 RH10),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3608565,22.140,False,False,99.209892,221.840062,2023-Sep-28 00:02,Earth,10.2005587063,36722.0113425569,0.2989174969,116.2789062941,44717420.841971603,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3648878,(2013 TG6),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3648878,26.600,False,False,12.721988,28.447230,2023-Sep-28 21:33,Earth,4.1477046929,14931.7368946125,0.0091324629,3.5525280681,1366196.997694023,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3690061,(2014 SH144),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3690061,25.000,False,False,26.580000,59.434687,2023-Sep-28 21:46,Earth,12.5398583222,45143.4899599524,0.0956959945,37.2257418605,14315916.944731715,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,54202343,(2021 TQ1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54202343,28.090,False,False,6.405529,14.323197,2023-Sep-21 05:58,Earth,20.3897653573,73403.1552863347,0.4971154154,193.3778965906,74367407.288005198,{'links': {'self': 'http://api.nasa.gov/neo/re...
89,54244535,(2022 CT),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54244535,25.730,False,False,18.991312,42.465865,2023-Sep-21 06:28,Earth,8.5561768174,30802.2365427628,0.4900134095,190.6152162955,73304962.332637765,{'links': {'self': 'http://api.nasa.gov/neo/re...
90,54310136,(2022 SD29),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54310136,26.480,False,False,13.444820,30.063530,2023-Sep-21 05:47,Earth,10.5189116588,37868.0819718337,0.217857528,84.746578392,32591022.15226536,{'links': {'self': 'http://api.nasa.gov/neo/re...
91,54359399,(2023 KK1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54359399,20.200,True,False,242.412481,542.050786,2023-Sep-21 01:09,Earth,13.3678083073,48124.1099061374,0.1966111821,76.4817498369,29412614.060342127,{'links': {'self': 'http://api.nasa.gov/neo/re...


In [235]:
df = df.rename(columns={
    'estimated_diameter.meters.estimated_diameter_min': 'estimated_diameter.meters_min',
    'estimated_diameter.meters.estimated_diameter_max': 'estimated_diameter.meters_max'
    })
df

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter.meters_min,estimated_diameter.meters_max,close_approach_date_full,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,data
0,3384067,(2007 RU9),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3384067,20.700,True,False,192.555078,430.566244,2023-Sep-28 15:52,Earth,21.2534376859,76512.3756692011,0.1604399752,62.4111503528,24001478.552772824,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,3581506,(2011 TB4),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3581506,25.400,False,False,22.108281,49.435619,2023-Sep-28 21:43,Earth,19.6051964197,70578.7071110284,0.2179460329,84.7810067981,32604262.296789923,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3608565,(2012 RH10),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3608565,22.140,False,False,99.209892,221.840062,2023-Sep-28 00:02,Earth,10.2005587063,36722.0113425569,0.2989174969,116.2789062941,44717420.841971603,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3648878,(2013 TG6),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3648878,26.600,False,False,12.721988,28.447230,2023-Sep-28 21:33,Earth,4.1477046929,14931.7368946125,0.0091324629,3.5525280681,1366196.997694023,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3690061,(2014 SH144),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3690061,25.000,False,False,26.580000,59.434687,2023-Sep-28 21:46,Earth,12.5398583222,45143.4899599524,0.0956959945,37.2257418605,14315916.944731715,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,54202343,(2021 TQ1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54202343,28.090,False,False,6.405529,14.323197,2023-Sep-21 05:58,Earth,20.3897653573,73403.1552863347,0.4971154154,193.3778965906,74367407.288005198,{'links': {'self': 'http://api.nasa.gov/neo/re...
89,54244535,(2022 CT),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54244535,25.730,False,False,18.991312,42.465865,2023-Sep-21 06:28,Earth,8.5561768174,30802.2365427628,0.4900134095,190.6152162955,73304962.332637765,{'links': {'self': 'http://api.nasa.gov/neo/re...
90,54310136,(2022 SD29),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54310136,26.480,False,False,13.444820,30.063530,2023-Sep-21 05:47,Earth,10.5189116588,37868.0819718337,0.217857528,84.746578392,32591022.15226536,{'links': {'self': 'http://api.nasa.gov/neo/re...
91,54359399,(2023 KK1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54359399,20.200,True,False,242.412481,542.050786,2023-Sep-21 01:09,Earth,13.3678083073,48124.1099061374,0.1966111821,76.4817498369,29412614.060342127,{'links': {'self': 'http://api.nasa.gov/neo/re...


Adding date awareness to allow for updating data without duplicates and loading previous data

In [236]:
datetime.date.today() - datetime.timedelta(days=365)

datetime.date(2022, 9, 30)

In [271]:
try:
    with open('last_executed.txt') as f:
        last_execution_date = datetime.datetime.strptime(f.read(), '%Y-%m-%d').date()
except FileNotFoundError:
        last_execution_date = (datetime.date.today() - datetime.timedelta(days=365))

last_execution_date

datetime.date(2022, 9, 30)

In [274]:
delta = (datetime.date.today() - last_execution_date).days
delta

365

In [ ]:
# while delta % 7 != 0:
#     variable = (datetime.date.today() - datetime.timedelta(delta%7))
#     date = variable.strftime('%Y-%m-%d')

#     link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
    
#     delta = delta - delta % 7
#     print(link)
# while delta // 7 != 0:
#     variable = variable - datetime.timedelta(days=7)
#     date = variable.strftime('%Y-%m-%d')

#     link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
    
#     delta = delta - 7
#     print(link)

In [275]:
links_a = []
links_b = []

if delta % 7 != 0:
    end_date = datetime.date.today()
    start_date = (end_date - datetime.timedelta(days=delta%7)).strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
    links_a.append(link)

for x in range(delta%7, delta+7, 8):
    end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
    start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
    links_b.append(link)

links = links_a + links_b
links


['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-29&end_date=2023-09-30&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-21&end_date=2023-09-28&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-13&end_date=2023-09-20&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-05&end_date=2023-09-12&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-08-28&end_date=2023-09-04&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-08-20&end_date=2023-08-27&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-08-12&end_date=2023-08-19&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-08-04&end_dat

In [276]:
def load_data():
    
    for link in links:
    
        response = requests.get(link)
        data = response.json()
        neo_data = data['near_earth_objects']

        cursor = conn.cursor()

        cursor.executescript('''

             CREATE TABLE IF NOT EXISTS neo (
                            id text,
                            data json
            )
        ''')

        for date, neo_list in neo_data.items():
            for neo in neo_list:
                cursor.execute('INSERT INTO neo VALUES (?, ?)',
                            (neo['id'], json.dumps(neo)))
        conn.commit()

        with open('last_executed.txt', 'w') as f:
            f.write(str(datetime.date.today()))

In [277]:
from main import transform

load_data(), transform()

c:\Users\Jack\Desktop\Coding\NASA\main.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}


            id          name                                    nasa_jpl_url  \
0      3384067    (2007 RU9)   http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3384067   
1      3581506    (2011 TB4)   http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3581506   
2      3608565   (2012 RH10)   http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3608565   
3      3648878    (2013 TG6)   http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3648878   
4      3690061  (2014 SH144)   http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3690061   
...        ...           ...                                             ...   
9254  54305672    (2022 SK5)  http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54305672   
9255  54308110   (2022 SF19)  http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54308110   
9256  54311855   (2022 SL48)  http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54311855   
9257  54312937   (2022 SW55)  http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54312937   
9258  54312822     (2022 TC)  http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54312822   

      absolute_magnitude_h  is_potentia

(None, None)